In [2]:
import pandas as pd

In [3]:
in_path = r'../datos/'
out_path = r'../datos/'
dfile = 'datos_temperaturas.xlsx'

In [4]:
df = pd.read_excel(in_path + dfile, header=1)
df.head()

,fecha,mes,temp
0,2000,ENE,5.15
1,2000,FEB,4.90
2,2000,MAR,4.65
3,2000,ABR,4.05
4,2000,MAY,2.40


In [5]:
lat = pd.read_excel(in_path + dfile, usecols =[1], nrows=1, header=None)[1][0]
lat

-17.094167

In [6]:
df.loc[df['temp'] <0, 'temp'].count()

6

In [7]:
df.loc[df['temp'] <=0, 'temp'] = 0
df.loc[df['temp'] <0, 'temp'].count()

0

### Cálculo indice calor mensual

## $$i=(\frac{t}{5})^{1.514}$$
#### i = Índice de calor mensual
#### t = temperatura promedio mensual

In [8]:
df["ind_calor"] = (df["temp"]/5)**1.154
df.head()

,fecha,mes,temp,ind_calor
0,2000,ENE,5.15,1.034699
1,2000,FEB,4.90,0.976956
2,2000,MAR,4.65,0.919664
3,2000,ABR,4.05,0.784137
4,2000,MAY,2.40,0.428699


### Cálculo indice calor anual

## $$ I = \sum{i}$$

In [9]:
df["i_anual"] = [0.0 for x in range(len(df))]
df.head()

,fecha,mes,temp,ind_calor,i_anual
0,2000,ENE,5.15,1.034699,0.0
1,2000,FEB,4.90,0.976956,0.0
2,2000,MAR,4.65,0.919664,0.0
3,2000,ABR,4.05,0.784137,0.0
4,2000,MAY,2.40,0.428699,0.0


In [10]:
for i in range(len(df)):
    df.loc[i, 'i_anual'] = df.loc[df['fecha'] == df["fecha"][i],'temp'].sum()
df.head()

,fecha,mes,temp,ind_calor,i_anual
0,2000,ENE,5.15,1.034699,42.9
1,2000,FEB,4.90,0.976956,42.9
2,2000,MAR,4.65,0.919664,42.9
3,2000,ABR,4.05,0.784137,42.9
4,2000,MAY,2.40,0.428699,42.9


In [11]:
df.tail()

,fecha,mes,temp,ind_calor,i_anual
199,2016,AGO,1.50,0.249229,44.1
200,2016,SEP,3.20,0.597491,44.1
201,2016,OCT,4.70,0.931085,44.1
202,2016,NOV,5.75,1.175020,44.1
203,2016,DIC,5.80,1.186819,44.1


### Cálculo ETP sin corr Método Thornthwaite

## $$ ETP_{sin/corr} = 16\cdot (\frac{10\cdot t}{I})^{a} \frac{mm}{mes}$$
### $$a = 6.75\cdot10^{-7}\cdot I^{3} - 7.71\cdot10^{-5}\cdot I^{2} + 0.01792\cdot I + 0.49239$$
#### ETP = Evapotranspiración sin corregir
#### t = temperatura promedio mensual
#### I = Índice de calor anual


In [12]:
def cte_a(I):
    return (6.75e-7 * I**3) - (7.71e-5 * I**2) + (0.01792 * I) + 0.49239

In [13]:
cte_a(23)

0.871976825

In [14]:
df["etp_sc"] = 16*( ( (10*df["temp"])/df["i_anual"] )**(cte_a(df["i_anual"])))
df

,fecha,mes,temp,ind_calor,i_anual,etp_sc
0,2000,ENE,5.15,1.034699,42.9,19.822674
1,2000,FEB,4.90,0.976956,42.9,18.699154
2,2000,MAR,4.65,0.919664,42.9,17.585485
3,2000,ABR,4.05,0.784137,42.9,14.955586
4,2000,MAY,2.40,0.428699,42.9,8.097433
...,...,...,...,...,...,...
199,2016,AGO,1.50,0.249229,44.1,4.430995
200,2016,SEP,3.20,0.597491,44.1,10.921484
201,2016,OCT,4.70,0.931085,44.1,17.260420
202,2016,NOV,5.75,1.175020,44.1,21.943859


In [16]:
df.to_excel('../resultados/etp.xlsx', index=False)

### Cargar datos Horas Sol

In [181]:
ds = pd.read_csv('../datos/horas_sol.csv', sep=';')
ds

,LAT 20,LAT 18,LAT 16,LAT 14
0,"13,1",13,"12,9","12,7"
1,"12,7","12,6","12,5","12,4"
2,"12,1","12,1","12,1","12,1"
3,"11,5","11,6","11,6","11,7"
4,"11,1","11,2","11,3","11,4"
5,"10,8","10,9","11,1","11,2"
6,"10,9",11,"11,1","11,2"
7,"11,3","11,4","11,5","11,5"
8,"11,9","11,9","11,9","11,9"
9,"12,5","12,4","12,4","12,3"


In [180]:
lat

-17.094167

### Interpolación

## $$ y = y_{1} + \frac{(y_{2}-y_{1})}{(x_{2}-x_{1})} \cdot (x-x_{1})$$

### $$TAREA$$

### Cálculo ETP corr

## $$ ETP_{corr} = ETP_{sin/corr}\cdot \frac{N}{12} \cdot \frac{d}{30} $$